# Detección de Componentes Electrónicos en PCB: YOLOv8 vs RT-DETR
## Proyecto de Deep Learning - Detección de Objetos

**Autor:** [Tu Nombre]
**Dataset:** Printed Circuit Board Defects - Roboflow Universe
**Modelos:** YOLOv8s, RT-DETR
**Fecha:** [Fecha de ejecución]

## 1. Setup e Instalación de Librerías

Instalamos todas las librerías necesarias para el proyecto.

In [ ]:
# Instalación de librerías principales
!pip install ultralytics roboflow transformers torch torchvision
!pip install opencv-python matplotlib seaborn pandas numpy
!pip install albumentations pillow requests tqdm

# Verificar GPU
import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Imports necesarios
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Ultralytics YOLO
from ultralytics import YOLO

# Transformers para RT-DETR
from transformers import RTDetrForObjectDetection, RTDetrImageProcessor

# Torch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Roboflow
from roboflow import Roboflow

# Configuración
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

print("✅ Todas las librerías importadas correctamente")

## 2. Carga y Exploración del Dataset PCB

Descargamos y exploramos el dataset de defectos en placas de circuito impreso desde Roboflow.

In [ ]:
# Configuración de Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")  # Reemplazar con tu API key

# Descargar dataset PCB
project = rf.workspace("roboflow-100").project("printed-circuit-board")
dataset = project.version(1).download("yolov8")

# Mostrar estructura del dataset
dataset_path = dataset.location
print(f"Dataset descargado en: {dataset_path}")

# Explorar estructura
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Mostrar solo primeros 5 archivos
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... y {len(files)-5} archivos más")

In [ ]:
# Análisis del dataset
data_yaml_path = os.path.join(dataset_path, 'data.yaml')

# Leer configuración del dataset
import yaml
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("📊 INFORMACIÓN DEL DATASET")
print("=" * 40)
print(f"Número de clases: {data_config['nc']}")
print(f"Clases: {data_config['names']}")
print(f"Ruta train: {data_config['train']}")
print(f"Ruta val: {data_config['val']}")
print(f"Ruta test: {data_config['test']}")

# Contar imágenes en cada split
train_images = len(os.listdir(os.path.join(dataset_path, 'train', 'images')))
val_images = len(os.listdir(os.path.join(dataset_path, 'valid', 'images')))
test_images = len(os.listdir(os.path.join(dataset_path, 'test', 'images')))

print(f"
📈 DISTRIBUCIÓN DE DATOS")
print("=" * 40)
print(f"Imágenes de entrenamiento: {train_images}")
print(f"Imágenes de validación: {val_images}")
print(f"Imágenes de prueba: {test_images}")
print(f"Total: {train_images + val_images + test_images}")

# Guardar información para uso posterior
dataset_info = {
    'path': dataset_path,
    'classes': data_config['names'],
    'nc': data_config['nc'],
    'train_size': train_images,
    'val_size': val_images,
    'test_size': test_images
}

In [ ]:
# Visualizar muestras del dataset
def plot_sample_images_with_labels(dataset_path, split='train', num_samples=6):
    """Visualiza imágenes de muestra con sus etiquetas"""
    
    images_dir = os.path.join(dataset_path, split, 'images')
    labels_dir = os.path.join(dataset_path, split, 'labels')
    
    image_files = os.listdir(images_dir)[:num_samples]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    class_names = data_config['names']
    colors = plt.cm.Set3(np.linspace(0, 1, len(class_names)))
    
    for idx, img_file in enumerate(image_files):
        # Cargar imagen
        img_path = os.path.join(images_dir, img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        # Cargar etiquetas
        label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(labels_dir, label_file)
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            # Dibujar bounding boxes
            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])
                x_center, y_center, width, height = map(float, parts[1:5])
                
                # Convertir a coordenadas de píxeles
                x1 = int((x_center - width/2) * w)
                y1 = int((y_center - height/2) * h)
                x2 = int((x_center + width/2) * w)
                y2 = int((y_center + height/2) * h)
                
                # Dibujar rectángulo
                color = colors[class_id][:3]  # RGB
                color = tuple(int(c * 255) for c in color)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                
                # Añadir etiqueta
                label = class_names[class_id]
                cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 
                           0.5, color, 1)
        
        axes[idx].imshow(img)
        axes[idx].set_title(f"Imagen {idx+1}: {img_file}")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle(f"Muestras del conjunto de {split.upper()}", y=1.02, fontsize=16)
    plt.show()

# Visualizar muestras
plot_sample_images_with_labels(dataset_path, 'train', 6)

## 3. Preprocesamiento y Data Augmentation

Configuramos las transformaciones y augmentaciones para mejorar la robustez del modelo.

In [ ]:
# Configuración de augmentaciones usando Albumentations
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.Rotate(limit=15, p=0.4),
    A.RandomBrightnessContrast(
        brightness_limit=0.2, 
        contrast_limit=0.2, 
        p=0.5
    ),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.Blur(blur_limit=3, p=0.2),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

val_transforms = A.Compose([
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

print("✅ Configuración de augmentaciones completada")
print("
Transformaciones de entrenamiento:")
for transform in train_transforms.transforms:
    print(f"  - {transform.__class__.__name__}")

print("
Transformaciones de validación:")
for transform in val_transforms.transforms:
    print(f"  - {transform.__class__.__name__}")

In [ ]:
# Demostración de augmentaciones
def show_augmentation_effects(dataset_path, num_examples=3):
    """Muestra el efecto de las augmentaciones en imágenes de muestra"""
    
    images_dir = os.path.join(dataset_path, 'train', 'images')
    labels_dir = os.path.join(dataset_path, 'train', 'labels')
    
    image_files = os.listdir(images_dir)[:num_examples]
    
    fig, axes = plt.subplots(num_examples, 4, figsize=(16, 4*num_examples))
    
    for i, img_file in enumerate(image_files):
        # Cargar imagen original
        img_path = os.path.join(images_dir, img_file)
        original_img = cv2.imread(img_path)
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        
        # Cargar etiquetas (simplificado para demostración)
        label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(labels_dir, label_file)
        
        bboxes = []
        class_labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    class_labels.append(int(parts[0]))
                    bboxes.append([float(x) for x in parts[1:5]])
        
        # Mostrar imagen original
        axes[i, 0].imshow(original_img)
        axes[i, 0].set_title('Original')
        axes[i, 0].axis('off')
        
        # Aplicar diferentes augmentaciones
        aug_names = ['Flip + Rotate', 'Brightness/Contrast', 'Noise + Blur']
        
        for j in range(3):
            if bboxes:
                augmented = train_transforms(image=original_img.copy(), 
                                           bboxes=bboxes, 
                                           class_labels=class_labels)
                aug_img = augmented['image']
            else:
                aug_img = train_transforms(image=original_img.copy(), 
                                         bboxes=[], 
                                         class_labels=[])['image']
            
            # Desnormalizar para visualización
            aug_img = aug_img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
            aug_img = np.clip(aug_img, 0, 1)
            
            axes[i, j+1].imshow(aug_img)
            axes[i, j+1].set_title(aug_names[j])
            axes[i, j+1].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Efectos de Data Augmentation', y=1.02, fontsize=16)
    plt.show()

# Mostrar efectos de augmentación
show_augmentation_effects(dataset_path, 2)

## 4. Entrenamiento de Modelos

Entrenamos tres variantes: YOLOv8s y RT-DETR con configuraciones estándar.

In [ ]:
# Entrenamiento YOLOv8s
print("🚀 INICIANDO ENTRENAMIENTO YOLOv8s")
print("=" * 50)

# Cargar modelo base
yolo_model = YOLO('yolov8s.pt')  # Carga el modelo preentrenado

# Configuración de entrenamiento
yolo_results = yolo_model.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640,
    batch=16,
    name='yolov8s_pcb',
    save_period=10,
    patience=15,
    optimizer='AdamW',
    lr0=0.001,
    weight_decay=0.0005,
    augment=True,
    mosaic=0.8,
    mixup=0.1,
    copy_paste=0.1,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=8,
    verbose=True,
    seed=42
)

print("✅ Entrenamiento YOLOv8s completado")

# Guardar ruta del modelo entrenado
yolo_model_path = yolo_model.trainer.best

In [ ]:
# Configuración RT-DETR
print("🚀 CONFIGURANDO RT-DETR")
print("=" * 50)

# Nota: RT-DETR también puede entrenarse usando Ultralytics
# que simplifica enormemente el proceso

# Cargar RT-DETR usando ultralytics
rtdetr_model = YOLO('rtdetr-l.pt')  # Carga RT-DETR preentrenado

print("✅ Modelo RT-DETR cargado")

In [ ]:
# Entrenamiento RT-DETR
print("🚀 INICIANDO ENTRENAMIENTO RT-DETR")
print("=" * 50)

# Configuración de entrenamiento para RT-DETR
rtdetr_results = rtdetr_model.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640,
    batch=8,  # Batch menor por ser un modelo más pesado
    name='rtdetr_pcb',
    save_period=10,
    patience=15,
    optimizer='AdamW',
    lr0=0.0001,  # Learning rate más bajo para transformers
    weight_decay=0.0001,
    augment=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=4,
    verbose=True,
    seed=42
)

print("✅ Entrenamiento RT-DETR completado")

# Guardar ruta del modelo entrenado
rtdetr_model_path = rtdetr_model.trainer.best

## 5. Evaluación y Métricas

Evaluamos todos los modelos con múltiples métricas: mAP@0.5, mAP@[0.5:0.95], Precision por clase, Recall por clase, y velocidad de inferencia.

In [ ]:
# Evaluación completa de modelos
def evaluate_model(model_path, model_name, data_yaml):
    """Evalúa un modelo entrenado y retorna métricas detalladas"""
    
    print(f"
📊 EVALUANDO {model_name}")
    print("=" * 50)
    
    # Cargar modelo entrenado
    model = YOLO(model_path)
    
    # Validación en conjunto de test
    results = model.val(
        data=data_yaml,
        split='test',
        imgsz=640,
        batch=16,
        verbose=True,
        save_json=True,
        name=f'{model_name}_eval'
    )
    
    # Extraer métricas
    metrics = {
        'model': model_name,
        'mAP_50': results.box.map50,
        'mAP_50_95': results.box.map,
        'precision': results.box.mp,
        'recall': results.box.mr,
        'f1_score': 2 * (results.box.mp * results.box.mr) / (results.box.mp + results.box.mr) if (results.box.mp + results.box.mr) > 0 else 0
    }
    
    # Métricas por clase
    class_metrics = {
        'precision_per_class': results.box.p,
        'recall_per_class': results.box.r,
        'ap50_per_class': results.box.ap50,
        'ap_per_class': results.box.ap
    }
    
    return metrics, class_metrics, results

# Evaluación de velocidad de inferencia
def benchmark_inference_speed(model_path, dataset_path, num_images=100):
    """Mide la velocidad de inferencia del modelo"""
    
    model = YOLO(model_path)
    test_images_dir = os.path.join(dataset_path, 'test', 'images')
    test_images = os.listdir(test_images_dir)[:num_images]
    
    times = []
    
    print(f"⏱️  Midiendo velocidad de inferencia en {len(test_images)} imágenes...")
    
    for img_file in test_images:
        img_path = os.path.join(test_images_dir, img_file)
        
        # Medir tiempo de inferencia
        start_time = time.time()
        results = model(img_path, verbose=False)
        end_time = time.time()
        
        times.append(end_time - start_time)
    
    avg_time = np.mean(times)
    fps = 1.0 / avg_time
    
    return {
        'avg_inference_time': avg_time,
        'fps': fps,
        'min_time': np.min(times),
        'max_time': np.max(times),
        'std_time': np.std(times)
    }

print("✅ Funciones de evaluación definidas")

In [ ]:
# Ejecutar evaluaciones
import time

# Diccionario para almacenar todas las métricas
all_metrics = {}
all_class_metrics = {}
inference_speeds = {}

# Lista de modelos para evaluar
models_to_evaluate = [
    (yolo_model_path, 'YOLOv8s'),
    (rtdetr_model_path, 'RT-DETR-L')
]

# Evaluar cada modelo
for model_path, model_name in models_to_evaluate:
    # Métricas de precisión
    metrics, class_metrics, results = evaluate_model(model_path, model_name, data_yaml_path)
    all_metrics[model_name] = metrics
    all_class_metrics[model_name] = class_metrics
    
    # Velocidad de inferencia
    speed_metrics = benchmark_inference_speed(model_path, dataset_path)
    inference_speeds[model_name] = speed_metrics
    
    print(f"
✅ {model_name} evaluado completamente")

print("
🎉 TODAS LAS EVALUACIONES COMPLETADAS")
print("=" * 50)

In [ ]:
# Mostrar resultados tabulados
def display_comprehensive_results(all_metrics, inference_speeds, class_names):
    """Muestra resultados completos de evaluación"""
    
    print("
📊 RESUMEN COMPLETO DE RESULTADOS")
    print("=" * 80)
    
    # Crear DataFrame con métricas generales
    results_data = []
    
    for model_name in all_metrics.keys():
        row = {
            'Modelo': model_name,
            'mAP@0.5': f"{all_metrics[model_name]['mAP_50']:.3f}",
            'mAP@0.5:0.95': f"{all_metrics[model_name]['mAP_50_95']:.3f}",
            'Precision': f"{all_metrics[model_name]['precision']:.3f}",
            'Recall': f"{all_metrics[model_name]['recall']:.3f}",
            'F1-Score': f"{all_metrics[model_name]['f1_score']:.3f}",
            'FPS': f"{inference_speeds[model_name]['fps']:.1f}",
            'Tiempo (ms)': f"{inference_speeds[model_name]['avg_inference_time']*1000:.1f}"
        }
        results_data.append(row)
    
    results_df = pd.DataFrame(results_data)
    print(results_df.to_string(index=False))
    
    # Métricas por clase
    print("

📈 MÉTRICAS POR CLASE")
    print("=" * 50)
    
    for model_name in all_metrics.keys():
        print(f"
{model_name}:")
        print("-" * 30)
        
        class_data = []
        for i, class_name in enumerate(class_names):
            if i < len(all_class_metrics[model_name]['precision_per_class']):
                row = {
                    'Clase': class_name,
                    'Precision': f"{all_class_metrics[model_name]['precision_per_class'][i]:.3f}",
                    'Recall': f"{all_class_metrics[model_name]['recall_per_class'][i]:.3f}",
                    'AP@0.5': f"{all_class_metrics[model_name]['ap50_per_class'][i]:.3f}",
                    'AP@0.5:0.95': f"{all_class_metrics[model_name]['ap_per_class'][i]:.3f}"
                }
                class_data.append(row)
        
        class_df = pd.DataFrame(class_data)
        print(class_df.to_string(index=False))
    
    return results_df

# Mostrar resultados
results_table = display_comprehensive_results(all_metrics, inference_speeds, data_config['names'])

## 6. Visualización de Resultados

Creamos gráficas comparativas de métricas por modelo.

In [ ]:
# Gráficas comparativas de métricas
def plot_model_comparison(all_metrics, inference_speeds):
    """Crea gráficas comparativas entre modelos"""
    
    models = list(all_metrics.keys())
    
    # Preparar datos
    metrics_data = {
        'mAP@0.5': [all_metrics[model]['mAP_50'] for model in models],
        'mAP@0.5:0.95': [all_metrics[model]['mAP_50_95'] for model in models],
        'Precision': [all_metrics[model]['precision'] for model in models],
        'Recall': [all_metrics[model]['recall'] for model in models],
        'F1-Score': [all_metrics[model]['f1_score'] for model in models]
    }
    
    # Crear subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Comparación de Rendimiento entre Modelos', fontsize=16, fontweight='bold')
    
    # Colores para los modelos
    colors = ['#2E86AB', '#A23B72', '#F18F01']
    
    # Gráfica de barras para métricas principales
    metric_names = list(metrics_data.keys())
    x = np.arange(len(models))
    width = 0.15
    
    for i, metric in enumerate(metric_names):
        ax = axes[i//3, i%3]
        bars = ax.bar(x, metrics_data[metric], width*3, color=colors[:len(models)], alpha=0.8)
        
        ax.set_xlabel('Modelos')
        ax.set_ylabel(metric)
        ax.set_title(f'{metric} por Modelo')
        ax.set_xticks(x)
        ax.set_xticklabels(models, rotation=45, ha='right')
        ax.grid(True, alpha=0.3)
        
        # Añadir valores en las barras
        for bar, value in zip(bars, metrics_data[metric]):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                   f'{value:.3f}', ha='center', va='bottom', fontsize=10)
    
    # Gráfica de velocidad de inferencia
    ax = axes[1, 2]
    fps_values = [inference_speeds[model]['fps'] for model in models]
    bars = ax.bar(models, fps_values, color=colors[:len(models)], alpha=0.8)
    
    ax.set_xlabel('Modelos')
    ax.set_ylabel('FPS')
    ax.set_title('Velocidad de Inferencia (FPS)')
    ax.grid(True, alpha=0.3)
    
    for bar, value in zip(bars, fps_values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
               f'{value:.1f}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Gráfico radar para comparación multidimensional
    plot_radar_comparison(all_metrics, inference_speeds)

def plot_radar_comparison(all_metrics, inference_speeds):
    """Crea un gráfico radar para comparación multidimensional"""
    
    models = list(all_metrics.keys())
    
    # Normalizar métricas para el radar
    metrics_for_radar = ['mAP_50', 'mAP_50_95', 'precision', 'recall', 'f1_score']
    labels = ['mAP@0.5', 'mAP@0.5:0.95', 'Precision', 'Recall', 'F1-Score']
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    # Configurar ángulos
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]  # Cerrar el círculo
    
    colors = ['#2E86AB', '#A23B72', '#F18F01']
    
    for i, model in enumerate(models):
        values = [all_metrics[model][metric] for metric in metrics_for_radar]
        values += values[:1]  # Cerrar el círculo
        
        ax.plot(angles, values, 'o-', linewidth=2, label=model, color=colors[i])
        ax.fill(angles, values, alpha=0.25, color=colors[i])
    
    # Configuración del gráfico
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)
    ax.set_ylim(0, 1)
    ax.set_title('Comparación Multidimensional de Modelos\n(Gráfico Radar)', 
                 size=16, fontweight='bold', pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    ax.grid(True)
    
    plt.show()

# Ejecutar visualizaciones
plot_model_comparison(all_metrics, inference_speeds)

In [ ]:
# Gráficas de métricas por clase
def plot_per_class_metrics(all_class_metrics, class_names):
    """Crea gráficas de métricas por clase para cada modelo"""
    
    models = list(all_class_metrics.keys())
    metrics = ['precision_per_class', 'recall_per_class', 'ap50_per_class']
    metric_labels = ['Precision por Clase', 'Recall por Clase', 'AP@0.5 por Clase']
    
    fig, axes = plt.subplots(len(metrics), 1, figsize=(12, 15))
    fig.suptitle('Métricas por Clase y Modelo', fontsize=16, fontweight='bold')
    
    colors = ['#2E86AB', '#A23B72', '#F18F01']
    x = np.arange(len(class_names))
    width = 0.25
    
    for metric_idx, (metric, label) in enumerate(zip(metrics, metric_labels)):
        ax = axes[metric_idx]
        
        for model_idx, model in enumerate(models):
            if metric in all_class_metrics[model] and len(all_class_metrics[model][metric]) >= len(class_names):
                values = all_class_metrics[model][metric][:len(class_names)]
                bars = ax.bar(x + model_idx * width, values, width, 
                             label=model, color=colors[model_idx], alpha=0.8)
                
                # Añadir valores en las barras
                for bar, value in zip(bars, values):
                    height = bar.get_height()
                    if height > 0.01:  # Solo mostrar si el valor es significativo
                        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                               f'{value:.2f}', ha='center', va='bottom', 
                               fontsize=8, rotation=90)
        
        ax.set_xlabel('Clases')
        ax.set_ylabel(label.split(' ')[0])
        ax.set_title(label)
        ax.set_xticks(x + width)
        ax.set_xticklabels(class_names, rotation=45, ha='right')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Crear gráficas por clase
plot_per_class_metrics(all_class_metrics, data_config['names'])

## 7. Inferencia en Imágenes de Prueba

Realizamos inferencia en 3-5 imágenes de test con bounding boxes, clases y scores.

In [ ]:
# Función para realizar inferencia y visualizar resultados
def run_inference_comparison(models_dict, dataset_path, num_images=5):
    """Ejecuta inferencia con múltiples modelos y compara resultados"""
    
    test_images_dir = os.path.join(dataset_path, 'test', 'images')
    test_images = os.listdir(test_images_dir)[:num_images]
    
    for img_idx, img_file in enumerate(test_images):
        img_path = os.path.join(test_images_dir, img_file)
        
        print(f"
🔍 INFERENCIA EN IMAGEN {img_idx + 1}: {img_file}")
        print("=" * 60)
        
        # Cargar imagen original
        original_img = cv2.imread(img_path)
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        
        # Crear subplot para comparación
        num_models = len(models_dict) + 1  # +1 for original
        fig, axes = plt.subplots(1, num_models, figsize=(6*num_models, 6))
        
        # Mostrar imagen original
        axes[0].imshow(original_img)
        axes[0].set_title('Imagen Original')
        axes[0].axis('off')
        
        # Ejecutar inferencia con cada modelo
        for model_idx, (model_name, model_path) in enumerate(models_dict.items(), 1):
            # Cargar modelo
            model = YOLO(model_path)
            
            # Realizar predicción
            results = model(img_path, verbose=False)
            
            # Dibujar resultados
            annotated_img = results[0].plot()
            annotated_img = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
            
            axes[model_idx].imshow(annotated_img)
            axes[model_idx].set_title(f'{model_name}')
            axes[model_idx].axis('off')
            
            # Mostrar detalles de detecciones
            print(f"
{model_name}:")
            print("-" * 30)
            
            if len(results[0].boxes) > 0:
                for box_idx, box in enumerate(results[0].boxes):
                    # Extraer información
                    conf = float(box.conf)
                    cls = int(box.cls)
                    class_name = data_config['names'][cls]
                    bbox = box.xyxy[0].tolist()
                    
                    print(f"  Detección {box_idx + 1}:")
                    print(f"    Clase: {class_name}")
                    print(f"    Confianza: {conf:.3f}")
                    print(f"    BBox: [{bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f}]")
                    
                    # [ESPACIO PARA IoU - Se calculará cuando ejecutes]
                    print(f"    IoU con GT: [COMPLETAR AL EJECUTAR]")
                    print()
            else:
                print("  No se detectaron objetos")
        
        plt.tight_layout()
        plt.show()
        
        # [ESPACIO PARA CAPTURAS - Guardar al ejecutar]
        print("
📸 [ESPACIO PARA CAPTURAS - Guardar imagen al ejecutar]")
        

# Diccionario con modelos entrenados
trained_models = {
    'YOLOv8s': yolo_model_path,
    'RT-DETR-L': rtdetr_model_path
}

# Ejecutar inferencia comparativa
run_inference_comparison(trained_models, dataset_path, 5)

In [ ]:
# Función para calcular IoU con ground truth
def calculate_iou_with_gt(pred_boxes, gt_boxes):
    """Calcula IoU entre predicciones y ground truth"""
    
    def box_iou(box1, box2):
        # box format: [x1, y1, x2, y2]
        # Calcular intersección
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        if x2 <= x1 or y2 <= y1:
            return 0.0
        
        intersection = (x2 - x1) * (y2 - y1)
        
        # Calcular área de ambos boxes
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        
        # Calcular unión
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0
    
    ious = []
    for pred_box in pred_boxes:
        max_iou = 0.0
        for gt_box in gt_boxes:
            iou = box_iou(pred_box, gt_box)
            max_iou = max(max_iou, iou)
        ious.append(max_iou)
    
    return ious

print("✅ Función de cálculo de IoU definida")
print("
💡 NOTA: Ejecuta las celdas anteriores para ver los resultados de inferencia")

## 8. Guardado de Resultados

Guardamos todos los resultados en la carpeta /results para análisis posterior.

In [ ]:
# Crear carpeta de resultados
results_dir = '/content/results'
os.makedirs(results_dir, exist_ok=True)

# Guardar métricas en CSV
results_table.to_csv(os.path.join(results_dir, 'metricas_comparativas.csv'), index=False)

# Guardar métricas detalladas en JSON
import json

detailed_results = {
    'general_metrics': all_metrics,
    'class_metrics': {model: {k: v.tolist() if hasattr(v, 'tolist') else v 
                              for k, v in metrics.items()} 
                      for model, metrics in all_class_metrics.items()},
    'inference_speeds': inference_speeds,
    'dataset_info': dataset_info
}

with open(os.path.join(results_dir, 'resultados_detallados.json'), 'w') as f:
    json.dump(detailed_results, f, indent=2)

# Crear README de resultados
readme_content = f"""
# Resultados del Proyecto de Detección PCB

## Resumen de Archivos

- `metricas_comparativas.csv`: Tabla comparativa de métricas principales
- `resultados_detallados.json`: Métricas completas y metadatos
- `inferencias/`: Capturas de inferencias en imágenes de test
- `graficas/`: Visualizaciones de comparación de modelos

## Modelos Evaluados

1. **YOLOv8s**: Modelo YOLO optimizado para velocidad
2. **RT-DETR-L**: Detector basado en transformers en tiempo real

## Dataset

- **Fuente**: Roboflow Universe - Printed Circuit Board
- **Clases**: {len(data_config['names'])} ({', '.join(data_config['names'])})
- **Total de imágenes**: {dataset_info['train_size'] + dataset_info['val_size'] + dataset_info['test_size']}
  - Entrenamiento: {dataset_info['train_size']}
  - Validación: {dataset_info['val_size']}
  - Prueba: {dataset_info['test_size']}

## Mejores Resultados

[COMPLETAR AL EJECUTAR - Aquí se mostrarán los mejores resultados obtenidos]

Generado el: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

with open(os.path.join(results_dir, 'README.md'), 'w') as f:
    f.write(readme_content)

# Crear subdirectorio para inferencias
os.makedirs(os.path.join(results_dir, 'inferencias'), exist_ok=True)
os.makedirs(os.path.join(results_dir, 'graficas'), exist_ok=True)

print("✅ Resultados guardados exitosamente en /content/results/")
print(f"📁 Archivos generados:")
for file in os.listdir(results_dir):
    print(f"  - {file}")

# Copiar a Google Drive si está montado
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import shutil
    gdrive_path = '/content/drive/MyDrive/PCB_Detection_Results'
    shutil.copytree(results_dir, gdrive_path, dirs_exist_ok=True)
    print(f"
☁️  Resultados también guardados en Google Drive: {gdrive_path}")
except:
    print(f"
💾 Para guardar en Google Drive, monta tu drive primero")

print(f"
🎯 PROYECTO COMPLETADO EXITOSAMENTE!")
print(f"📊 Revisa los archivos en /content/results/ para análisis detallado")

## 9. Conclusiones y Próximos Pasos

### Resumen de Resultados

[COMPLETAR AL EJECUTAR - Aquí se incluirán las conclusiones basadas en los resultados obtenidos]

### Modelo Ganador

[COMPLETAR AL EJECUTAR - Modelo con mejor rendimiento general]

### Recomendaciones

1. **Para aplicaciones en tiempo real**: Considerar el modelo con mejor FPS
2. **Para máxima precisión**: Usar el modelo con mejor mAP
3. **Para deployment en edge**: Evaluar el balance precisión-velocidad

### Trabajo Futuro

- Probar con más épocas de entrenamiento
- Experimentar con diferentes augmentaciones
- Evaluar modelos de diferentes tamaños (nano, medium, large)
- Implementar ensemble de modelos
- Optimizar para deployment (TensorRT, ONNX)

---

**¡Proyecto completado exitosamente!** 🎉

Todos los resultados, gráficas y análisis están guardados en `/content/results/` para revisión detallada.